In [11]:
import pandas as pd
import numpy as np

# Read
df = pd.read_csv('House_Babylon/House_Babylon.csv')

# Transform
df.columns = df.iloc[0]
df = df.drop([0, 1]).reset_index(drop=True)
df = df[["Item (EN)", "Description (EN)","Category/Department (EN)","Variant Name"]]

df =  df.head(5)

## **Model: Ollama 3.1**

In [12]:
import requests

def ollama_local(prompt):
    url = "http://127.0.0.1:11434/v1/completions"
    payload = {
        "model": "llama3.1:8b",
        "prompt": prompt,
        "max_tokens": 200,
        "stream": False
    }
    headers = {"Content-Type": "application/json"}

    response = requests.post(url, json=payload, headers=headers)
    response.raise_for_status()
    data = response.json()
    
    # Extract only the generated text
    return data["choices"][0]["text"].strip()


## **Level 1: Shopping Categories**

In [13]:
shopping_categories = [
    "stationary","restaurants","electronics","pharmacies","pet care","home and garden",
    "beauty","entertainment","health and nutrition","groceries","fashion",
    "automotive","sports","kids","flowers and gifts"
]

def classify_row(row):
    text = f"""
    Item: {row['Item (EN)']}
    Description: {row['Description (EN)']}
    Vendor Category: {row['Category/Department (EN)']}

    Task: Choose the best suited shopping category for this item.

    Allowed categories:
    {shopping_categories}

    Rules:
    - Return ONLY one category name exactly as written in the list
    - If none fit, return nothing
    - Do not add quotes, punctuation, or explanations
    """
    result = ollama_local(text).strip()
    
    # clean result
    result = result.lower().replace("'", "").replace('"', "").replace(":", "").strip()
    
    # validate against allowed categories
    if result not in shopping_categories:
        return ""  # leave empty if not valid
    
    return result

df["shopping_category"] = df.apply(classify_row, axis=1)


## Phi4:latest

In [14]:
import requests

API_URL = "http://100.75.237.4:11434/api/generate"

MISTRAL_API_MODEL = "mistral"
PHI4_API_MODEL = "phi4:latest"

def run_model(api_model, prompt: str):
    payload = {
        "model": api_model,
        "prompt": prompt,
        "stream": False
    }
    response = requests.post(API_URL, json=payload)
    response.raise_for_status()
    return response.json().get("response", "").strip()

In [15]:
# Test 1: Mistral
print("Mistral Output:")
print(run_model(MISTRAL_API_MODEL, "hi"))

Mistral Output:
Hello! How can I assist you today?

I am here to help answer questions, provide information, or just chat if you'd like. What would you like to talk about or know more about? I am happy to help with a wide range of topics.


In [16]:
# Test 2: Phi4
print("\nPhi4 Output:")
print(run_model(PHI4_API_MODEL, "hi"))


Phi4 Output:
Hello! How can I assist you today? If you have any questions or need information, feel free to let me know. 😊


## **Level 2: Shopping SubCategories**

In [17]:

# Subcategories for each main category
fashion_subcategories = ["baby clothes","undergarments","accessories","casual wear","home decor","beach wear","outerwear","neckwear and scarf","formal wear","sports wear","swimwear","designer wear","sports","outdoor sports","footwear","martial arts","sleepwear","medical wear","kids wear","jewelry","religious wear","fitness and training","maternity","water sports","eyewear"]
beauty_subcategories = ["skincare","haircare","fragrances","cosmetics"]
home_subcategories = ["drinkware","home decor","hardware and home improvement","tableware","lighting","kitchenwear","gardening and outdoor","bakeware","outdoor sports","household products","storage and organization","appliances","home essentials","bed and bath","fitness and training","furniture","kitchenware"]
stationary_subcategory = ["office supplies","stationary","stationary supplies","arts and crafts","school supplies","stationary accessories"]
restaurants_subcategory = ["desserts","mexican","pizza","specialty foods","egyptian","burgers","korean","fast food","street food","restaurants","japanese","italian","cafes","mediterranean","vegan","salads","asian","vietnamese","lebanese","international","pasta","chinese","bakery and cakes","french","juices & drinks","thai","sandwiches","seafood","sushi","keto","grills","healthy","vegetarian","indian","american","middle eastern","juices and drinks"]
pharmacies_subcategory = ["sportswear","specialty foods","first aid and medical equipment","haircare","dietary supplements","men's care","vitamins","skincare","women's deodorant","medicine","eye care","dental care","incontinence","women's care"]
pet_care_subcategory = ["aquatic animals","horses","dogs","birds","small pet supplies","cats"]
sports_subcategory = ["outdoor sports","sports","footwear","team sports","winter sports","recreational activities","swimwear","fitness and training","accessories","water sports","martial arts","sportswear"]
entertainment_subcategory = ["books","gaming","music","musical instruments","toys and games","musical equipment"]
health_and_nutrition_subcategory = ["natural solutions","weight management","specialty foods","men's care","performance supplements","protein products","vitamins","dietary supplements"]
groceries_subcategory = ["salads","supermarkets","specialty foods","mini marts","bakery and cakes","desserts","cafes"]
automotive_subcategory = ["auto accessories","motorcycle care"]
kids_subcategory = ["baby safety products","baby care","baby furniture","kids furniture","baby travel","baby clothes","swimwear","kids furniture accessory","toys and games"]
flowers_and_gifts_subcategory = ["flowers", "gifts"]

In [18]:
# mapping of main categories to their subcategories
subcategory_map = {
    "fashion": fashion_subcategories,
    "beauty": beauty_subcategories,
    "home and garden": home_subcategories,
    "stationary": stationary_subcategory,
    "restaurants": restaurants_subcategory,
    "pharmacies": pharmacies_subcategory,
    "pet care": pet_care_subcategory,
    "sports": sports_subcategory,
    "entertainment": entertainment_subcategory,
    "health and nutrition": health_and_nutrition_subcategory,
    "groceries": groceries_subcategory,
    "gifts and flowers": flowers_and_gifts_subcategory,
    "automotive": automotive_subcategory,
    "kids": kids_subcategory,
    "flowers and gifts": flowers_and_gifts_subcategory
}


def classify_subcategory(row):
    main_cat = row["shopping_category"]

    # if no main category, return empty
    if not main_cat or main_cat not in subcategory_map:
        return ""

    allowed_subcats = subcategory_map[main_cat]

    text = f"""
    Item: {row['Item (EN)']}
    Description: {row['Description (EN)']}
    Vendor Category: {row['Category/Department (EN)']}
    Main Category: {main_cat}

    Task: Choose the best suited subcategory for this item.

    Allowed subcategories:
    {allowed_subcats}

    Rules:
    - Return ONLY one subcategory name exactly as written in the list
    - If none fit, return nothing
    - Do not add quotes, punctuation, or explanations
    """

    result = ollama_local(text).strip().lower()
    result = result.replace("'", "").replace('"', "").replace(":", "").strip()

    # validate result
    if result not in allowed_subcats:
        return ""

    return result

df["shopping_subcategory"] = df.apply(classify_subcategory, axis=1)


In [19]:
df

,Item (EN),Description (EN),Category/Department (EN),Variant Name,shopping_category,shopping_subcategory
0,Albergo Stripe Set - Off White,A summer uniform reimagined in breathable Egyp...,Poplin Sets,XS/XS,fashion,casual wear
1,Albergo Stripe Set - Off White,A summer uniform reimagined in breathable Egyp...,Poplin Sets,S/XS,fashion,beach wear
2,Albergo Stripe Set - Off White,A summer uniform reimagined in breathable Egyp...,Poplin Sets,M/XS,fashion,beach wear
3,Albergo Stripe Set - Off White,A summer uniform reimagined in breathable Egyp...,Poplin Sets,L/XS,fashion,beach wear
4,Albergo Stripe Set - Off White,A summer uniform reimagined in breathable Egyp...,Poplin Sets,XL/XS,fashion,beach wear


## **Level 3: Item Category**

In [20]:
# ========================
# Fashion Level 3 Items
# ========================
baby_clothes = ["onesie", "babygrow", "romper", "jumpsuit", "baby gown", "diaper cover", "bloomers", "diaper shirt", "baby leggings", "lap tee", "baby shoe", "baby sock", "baby mitten", "baby accessory"]
undergarments = ["underwear", "stocking", "sock"]
accessories = ["headwear", "hair accessory", "belt", "bag", "wallet", "glove", "hand fan", "umbrella", "watch", "jewelry box"]
casual_wear = ["top", "trousers", "skirt", "dress", "outfit", "jumpsuit"]
home_decor_fashion = ["jewelry box", "hand fan"]
beach_wear = ["top", "bathing cover", "swimsuit", "sandal", "slipper"]
outerwear = ["outerwear", "robe", "jacket", "coat"]
neckwear_and_scarf = ["neckwear and scarf"]
formal_wear = ["suit", "dress", "tie pin", "cufflink"]
sports_wear = ["top", "trousers", "sports shoe", "outfit"]
swimwear = ["swimsuit", "bathing cover"]
designer_wear = ["dress", "suit", "outfit"]
sports_fashion = ["sports shoe", "top", "trousers", "outfit"]
outdoor_sports_fashion = ["sports shoe", "boot", "outerwear", "top", "trousers"]
footwear = ["sandal", "slipper", "shoe", "boot", "sports shoe", "sock", "stocking", "shoe accessory"]
martial_arts = ["uniform", "robe", "belt"]
sleepwear = ["pajamas", "robe", "nightgown"]
medical_wear = ["uniform"]
kids_wear = ["top", "trousers", "skirt", "dress", "outfit", "baby shoe", "baby sock", "baby mitten"]
jewelry = ["necklace", "bracelet", "earring", "ring", "pendant", "broche", "tie pin", "cufflink", "head ornament", "jewelry set"]
religious_wear = ["islamic religious wear", "christian religious wear"]
fitness_and_training_fashion = ["top", "trousers", "sports shoe", "outfit"]
maternity = ["top", "dress", "trousers", "outfit"]
water_sports_fashion = ["swimsuit", "bathing cover"]
eyewear = ["glasses", "sunglasses"]


# ========================
# Beauty Level 3 Items
# ========================
skincare = ["foot care", "acne", "face soap", "face cleanser", "bath soap", "body moisturizer", "hand care", "face treatment", "skin whitening", "face scrub", "face mask", "bath salt", "body oil", "loofah and sponge", "anti-aging", "dark spot", "face toner", "hand soap", "bath cream", "face roller", "cotton", "eye treatment", "sunscreen", "face moisturizer", "body scrub", "shower gel", "skincare accessory", "skincare set"]
haircare = ["hair gel", "hair mousse", "hair wax", "hair spray", "hair cream", "hair conditioner", "hair mask", "hair brush and comb", "hair styling tool", "hair dye", "hair loss", "hair shampoo", "hair oil", "hair serum", "hair treatment", "haircare set"]
fragrances = ["body spray", "cologne", "perfume"]
cosmetics = ["eye make-up", "lip make-up", "face make-up", "body make-up", "nailcare", "cosmetic accessory", "make-up tool", "cosmetic set"]
# ========================
# Home Level 3 Items
# ========================
drinkware = ["cup", "wine glass", "beer glass", "shot glass", "brandy glass", "tumbler", "coaster", "jar", "trembleuse", "glass", "champagne flute", "sake cup", "cognac glass", "whisky glass", "teacup", "pitcher", "wine opener", "mug", "martini glass", "sherry glass", "margarita glass", "rummer", "beaker", "carafe", "flask", "straw", "drinkware accessory"]
home_decor = ["wallpaper", "candle", "tapestery", "picture frame", "home scent", "mirror", "clock", "vase", "wall art", "decorative plate", "incense", "potpurri", "home decor accessory"]
hardware_and_home_improvement = ["home tool", "measuring tool", "plumbing tool", "electrical tool"]
tableware = ["cutlery", "plate and bowl", "table linen"]
lighting = ["wall lighting", "lamp", "chandelier", "light bulb", "lighting accessory", "ceiling lighting", "floor lighting", "outdoor lighting", "underwater lighting", "lighting system"]
kitchenwear = ["cooking utensil", "cooking tool", "specialty cookware", "ovenware", "pot and pan", "kitchen accessory"]
gardening_and_outdoor = ["gardening tool", "gardening equipment", "gardening care", "pool equipment"]
bakeware = ["baking pan", "bakeware utensil", "bakeware accessory"]
outdoor_sports_home = ["pool equipment"]
household_products = ["cleaning tool", "cleaning product", "house supply"]
storage_and_organization = ["office storage and organization", "clothing storage and organization", "bathroom storage and organization", "bedroom storage and organization", "kitchen storage and organization", "outdoor storage and organization", "kids storage and organization", "garage storage and organization", "trash bin", "basket", "food container", "rack", "drawer", "shelf", "hook", "recycling"]
appliances = ["personal appliance", "kitchen appliance", "home appliance", "heating and cooling unit", "cleaning appliance"]
home_essentials = ["house supply", "trash bin", "basket", "rack", "food container"]
bed_and_bath = ["bath linen", "bathroom accessory", "bathroom hardware", "bedding"]
fitness_and_training_home = ["home tool", "measuring tool"]
furniture = ["bedroom furniture", "kitchen furniture", "bathroom furniture", "dining room furniture", "living room furniture", "office furniture", "outdoor furniture", "furniture accessory"]
kitchenware = ["cooking utensil", "cooking tool", "specialty cookware", "ovenware", "pot and pan", "kitchen accessory"]


# ========================
# Stationary Level 3 Items
# ========================
office_supplies = ["scissors", "tape", "glue", "stapler", "tack", "paper clip", "paper puncher", "sticky note", "sheet protector", "folder", "document holder", "desk organizer", "desk planner", "calendar", "agenda", "diary", "clip board", "cork board", "white board", "chalk board", "office supply accessories"]
stationary = ["pencil", "pen", "marker", "sharpener", "eraser", "notebook", "writing pad", "ruler", "bookmark"]
stationary_supplies = ["pencil case", "pencil holder", "book cover", "keychain", "stationary accessories"]
arts_and_crafts = ["arts and crafts set", "craft supply", "knitting", "sewing", "jewelry making", "painting", "drawing", "pottery", "sculpting", "basket making", "candle making", "doll making", "craft fabric", "floral arranging", "weaving", "print making"]
school_supplies = ["pencil", "pen", "marker", "eraser", "sharpener", "ruler", "notebook", "writing pad", "compass", "crayon", "chalk", "school supply accessories"]
stationary_accessories = ["pencil case", "pencil holder", "book cover", "bookmark", "keychain"]


# ========================
# Restaurants Level 3 Items
# ========================
desserts = ["sweet", "bakery", "dairy", "snacks", "baking"]
mexican = ["meat and poultry", "produce", "grain", "condiment", "dairy"]
pizza = ["bakery", "grain", "dairy", "meat and poultry", "condiment"]
specialty_foods = ["pantry", "condiment", "produce", "deli"]
egyptian = ["grain", "meat and poultry", "produce", "dairy"]
burgers = ["meat and poultry", "bakery", "condiment", "dairy", "produce"]
korean = ["meat and poultry", "seafood", "produce", "condiment", "grain"]
fast_food = ["meat and poultry", "bakery", "dairy", "snacks", "condiment"]
street_food = ["meat and poultry", "produce", "bakery", "snacks"]
restaurants = ["meat and poultry", "seafood", "bakery", "produce", "dairy", "grain", "beverage"]
japanese = ["seafood", "produce", "grain", "condiment", "deli"]
italian = ["pasta", "grain", "meat and poultry", "dairy", "condiment"]
cafes = ["bakery", "sweet", "beverage", "dairy"]
mediterranean = ["produce", "seafood", "meat and poultry", "grain", "condiment"]
vegan = ["produce", "grain", "dairy", "snacks"]
salads = ["produce", "condiment"]
asian = ["grain", "meat and poultry", "seafood", "produce", "condiment"]
vietnamese = ["produce", "meat and poultry", "seafood", "grain", "condiment"]
lebanese = ["grain", "produce", "meat and poultry", "condiment"]
international = ["grain", "meat and poultry", "seafood", "produce", "dairy", "snacks", "condiment"]
pasta = ["pasta", "grain", "condiment", "dairy"]
chinese = ["grain", "meat and poultry", "seafood", "produce", "condiment"]
bakery_and_cakes = ["bakery", "sweet", "baking", "dairy"]
french = ["bakery", "dairy", "meat and poultry", "condiment", "sweet"]
juices_and_drinks = ["beverage", "produce", "dairy"]
thai = ["produce", "meat and poultry", "seafood", "grain", "condiment"]
sandwiches = ["bakery", "meat and poultry", "deli", "condiment", "dairy"]
seafood = ["seafood", "produce", "condiment"]
sushi = ["seafood", "grain", "deli", "condiment"]
keto = ["meat and poultry", "seafood", "dairy", "produce"]
grills = ["meat and poultry", "seafood", "condiment"]
healthy = ["produce", "grain", "dairy"]
vegetarian = ["produce", "grain", "dairy"]
indian = ["grain", "produce", "meat and poultry", "condiment", "dairy"]
american = ["meat and poultry", "bakery", "dairy", "snacks", "condiment"]
middle_eastern = ["grain", "meat and poultry", "produce", "dairy", "condiment"]


# ========================
# Pharmacies Level 3 Items
# ========================
sportswear_pharmacy = []
specialty_foods_pharmacy = []
first_aid_and_medical_equipment = []
haircare_pharmacy = []
dietary_supplements_pharmacy = []
mens_care = []
vitamins_pharmacy = []
skincare_pharmacy = []
womens_deodorant = []
medicine = []
eye_care = []
dental_care = []
incontinence = []
womens_care = []

# ========================
# Pet Care Level 3 Items
# ========================
aquatic_animals = []
horses = []
dogs = []
birds = []
small_pet_supplies = []
cats = []

# ========================
# Sports Level 3 Items
# ========================
outdoor_sports = []
sports = []
footwear_sports = []
team_sports = []
winter_sports = []
recreational_activities = []
swimwear_sports = []
fitness_and_training_sports = []
accessories_sports = []
water_sports = []
martial_arts_sports = []
sportswear_sports = []

# ========================
# Entertainment Level 3 Items
# ========================
books = []
gaming = []
music = []
musical_instruments = []
toys_and_games = []
musical_equipment = []

# ========================
# Health & Nutrition Level 3 Items
# ========================
natural_solutions = []
weight_management = []
specialty_foods_health = []
mens_care_health = []
performance_supplements = []
protein_products = []
vitamins_health = []
dietary_supplements_health = []

# ========================
# Groceries Level 3 Items
# ========================
salads_groceries = []
supermarkets = []
specialty_foods_groceries = []
mini_marts = []
bakery_and_cakes_groceries = []
desserts_groceries = []
cafes_groceries = []

# ========================
# Automotive Level 3 Items
# ========================
auto_accessories = []
motorcycle_care = []

# ========================
# Kids Level 3 Items
# ========================
baby_safety_products = []
baby_care = []
baby_furniture = []
kids_furniture = []
baby_travel = []
baby_clothes_kids = []
swimwear_kids = []
kids_furniture_accessory = []
toys_and_games_kids = []

# ========================
# Flowers & Gifts Level 3 Items
# ========================
flowers = []
gifts = []


In [21]:
# ===========================
# Fashion Item Category
# ===========================
baby_clothes = ["onesie", "diaper shirt", "babygrow", "lap tee", "romper", "baby shoe", "jumpsuit", "baby sock", "baby gown", "baby mitten", "diaper cover", "baby leggings", "bloomers", "baby accessory"]
undergarments = ["undershirt", "nightgown", "bra", "camisole", "panty", "baby doll", "boxers", "slip", "briefs", "corset"]
accessories = ["belt", "bag", "headwear", "hair accessories", "glove", "umbrella", "wallet", "luggage", "briefcase", "trunk", "laptopcase", "hand fan", "watch"]
casual_wear = ["apron", "top", "trousers", "blouse", "t-shirt", "shirt", "vest", "pants", "shorts", "leggings", "skirt", "dress", "outfit", "skort"]
home_decor_fashion = []
beach_wear = ["bathing cover"]
outerwear = ["sweatshirt", "sweater", "tank top", "jacket", "coat", "overalls", "jeans", "kimono"]
neckwear_and_scarf = ["tie", "shawl", "ascot", "pashmina", "boa", "scarf", "handkerchief", "bandana"]
formal_wear = ["blazer", "suit", "uniform", "rope"]
sports_wear = ["sportswear"]
swimwear = ["swimsuit"]
designer_wear = ["designer wear"]
sports_fashion = ["sports shoe"]
outdoor_sports_fashion = []
footwear = ["sandals", "slippers", "boot", "sports shoe", "shoe", "stocking", "sock", "shoe accessory"]
martial_arts = []
sleepwear = ["pajamas"]
medical_wear = ["medical wear"]
kids_wear = []
jewelry = ["earrings", "necklace", "bracelet", "broche", "pendant", "cufflink", "head ornament", "jewelry box", "ring", "jewelry set", "tie pin"]
religious_wear = ["islamic religious wear", "christian religious wear"]
fitness_and_training_fashion = []
maternity = ["maternity"]
water_sports_fashion = []
eyewear = ["glasses", "sunglasses"]


# ===========================
# Beauty Item Category
# ===========================
skincare = [
    "foot care", "hand care", "loofah and sponge", "cotton", "hair removal",
    "face treatment", "anti-aging", "eye treatment", "acne", "skin whitening",
    "dark spot", "sunscreen", "face soap", "face scrub", "face toner",
    "face moisturizer", "face cleanser", "face mask", "hand soap",
    "body scrub", "bath soap", "bath salt", "bath cream", "shower gel",
    "body moisturizer", "body oil", "face roller", "skincare accessory",
    "injectables", "skincare set"
]

haircare = [
    "hair gel", "hair brush and comb", "hair mousse", "hair styling tool",
    "hair wax", "hair dye", "hair spray", "hair loss", "hair cream",
    "hair shampoo", "hair conditioner", "hair oil", "hair mask", "hair serum",
    "haircare set", "hair treatment"
]

fragrances = [
    "body spray", "cologne", "perfume"
]

cosmetics = [
    "eye make-up", "lip make-up", "face make-up", "nailcare", "cosmetics accessory",
    "make-up tool", "body make-up", "cosmetic set"
]


# ===========================
# Home Item Category
# ===========================
drinkware = [
    "cup", "glass", "mug", "wine glass", "champagne flute", "martini glass",
    "beer glass", "sake cup", "sherry glass", "shot glass", "cognac glass",
    "margarira glass", "brandy glass", "whisky glass", "rummer", "tumbler",
    "teacup", "beaker", "coaster", "pitcher", "carafe", "jar", "wine opener",
    "flask", "trembleuse", "straw", "drinkware accessory"
]

home_decor = [
    "wallpaper", "clock", "candle", "vase", "tapestery", "wall art",
    "picture frame", "decorative plate", "home scent", "incense", "mirror",
    "potpurri", "home decor accessory"
]

hardware_and_home_improvement = [
    "home tool", "measuring tool", "plumbing tool", "electrical tool",
    "power tool", "hand tool", "welding", "duct tape", "cord", "flashlight",
    "nail and screw", "fastener and snap", "padlock", "shelf support",
    "window hardware", "home automation device"
]

tableware = [
    "cutlery", "plate and bowl", "table linen"
]

lighting = [
    "wall lighting", "lamp", "chandlier", "light bulb", "underwater lighting",
    "lighting accessory", "ceiling lighting", "floor lighting", "outdoor lighting",
    "lighting system"
]

kitchenwear = [
    "cooking utensil", "cooking tool", "speciality cookwear", "ovenware",
    "pot and pan", "kitchen accessory"
]

gardening_and_outdoor = [
    "gardening tool", "gardening equipment", "gardening care", "pool equipment"
]

bakeware = [
    "baking pan", "bakeware utensil", "bakeware accessory"
]

outdoor_sports_home = []

household_products = [
    "cleaning tool", "cleaning products", "house supply"
]

storage_and_organization = [
    "office storage and organization", "clothing storage and organization",
    "bathroom storage and organization", "bedroom storage and organization",
    "kitchen storage and organization", "outdoor storage and organization",
    "kids storage and organization", "garage storage and organization"
]

appliances = [
    "personal appliances", "kitchen appliance", "home appliance",
    "heating and cooling unit", "cleaning appliance", "specialty appliance"
]

home_essentials = []

bed_and_bath = [
    "bath linen", "bath accessory", "bathroom hardware", "bedding"
]

fitness_and_training_home = []

furniture = [
    "bedroom furniture", "kitchen furniture", "bathroom furniture",
    "dining room furniture", "living room furniture", "office furniture",
    "outdoor furniture", "furniture accessory", "kids furniture"
]

kitchenware = [
    "french press", "percolator", "coffee pot", "tea pot", "tea strainer",
    "cookware set", "double boiler", "braiser", "saucier"
]


# ===========================
# Stationary Item Category
# ===========================
office_supplies = [
    "scissors", "sharpener", "desk organizer", "paper puncher",
    "notebook", "chalk board", "desk planner", "white board",
    "writing pad", "chalk", "stapler", "calendar", "sticky note",
    "folder", "crayon", "tack", "agenda", "sheet protector",
    "clip board", "photo album", "cork board", "office supply accessories"
]

stationary = [
    "stationary"
]

stationary_supplies = [
    "document holder", "tape", "eraser", "glue", "pencil", "pen",
    "paper clip", "marker", "compass", "book cover", "bookmark",
    "ruler"
]

arts_and_crafts = [
    "craft supply", "knitting", "sewing", "jewelry making",
    "painting", "drawing", "pottery", "sculpting",
    "basket making", "candle making", "doll making", "craft fabric",
    "floral arranging", "weaving", "print making",
    "arts and crafts set"
]

school_supplies = [
    "notebook", "writing pad", "chalk board", "chalk", "folder",
    "crayon", "pencil case", "clip board", "pencil holder",
    "cork board", "book cover", "bookmark", "school supply accessories", "ruler"
]

stationary_accessories = [
    "keychain", "pencil case", "pencil holder"
]


# ===========================
# Restaurants Item Category
# ===========================
desserts = []
mexican = []
pizza = []
specialty_foods = []
egyptian = []
burgers = []
korean = []
fast_food = []
street_food = []
restaurants = []
japanese = []
italian = []
cafes = []
mediterranean = []
vegan = []
salads = []
asian = []
vietnamese = []
lebanese = []
international = []
pasta = []
chinese = []
bakery_and_cakes = []
french = []
juices_and_drinks = []
thai = []
sandwiches = []
seafood = []
sushi = []
keto = []
grills = []
healthy = []
vegetarian = []
indian = []
american = []
middle_eastern = []

# ===========================
# Pharmacies Item Category
# ===========================
sportswear_pharmacy = []
specialty_foods_pharmacy = []
first_aid_and_medical_equipment = []
haircare_pharmacy = []
dietary_supplements_pharmacy = []
mens_care = []
vitamins_pharmacy = []
skincare_pharmacy = []
womens_deodorant = []
medicine = []
eye_care = []
dental_care = []
incontinence = []
womens_care = []

# ===========================
# Pet Care Item Category
# ===========================
aquatic_animals = []
horses = []
dogs = []
birds = []
small_pet_supplies = []
cats = []

# ===========================
# Sports Item Category
# ===========================
outdoor_sports = []
sports = []
footwear_sports = []
team_sports = []
winter_sports = []
recreational_activities = []
swimwear_sports = []
fitness_and_training_sports = []
accessories_sports = []
water_sports = []
martial_arts_sports = []
sportswear_sports = []

# ===========================
# Entertainment Item Category
# ===========================
books = []
gaming = []
music = []
musical_instruments = []
toys_and_games = []
musical_equipment = []

# ===========================
# Health & Nutrition Item Category
# ===========================
natural_solutions = []
weight_management = []
specialty_foods_health = []
mens_care_health = []
performance_supplements = []
protein_products = []
vitamins_health = []
dietary_supplements_health = []

# ===========================
# Groceries Item Category
# ===========================
salads_groceries = []
supermarkets = []
specialty_foods_groceries = []
mini_marts = []
bakery_and_cakes_groceries = []
desserts_groceries = []
cafes_groceries = []

# ===========================
# Automotive Item Category
# ===========================
auto_accessories = []
motorcycle_care = []

# ===========================
# Kids Item Category
# ===========================
baby_safety_products = []
baby_care = []
baby_furniture = []
kids_furniture = []
baby_travel = []
baby_clothes_kids = []
swimwear_kids = []
kids_furniture_accessory = []
toys_and_games_kids = []

# ===========================
# Flowers & Gifts Item Category
# ===========================
flowers = []
gifts = []


In [ ]:
skincare = ["foot care", "acne", "face soap", ]
haircare = ["hair gel", "hair mousse", "hair wax"]
fragrances = ["body spray", "cologne", "perfume"]
cosmetics = ["eye make-up", "lip make-up", "face make-up"]